
#### NCSU GIS 714: Geospatial Computation and Simulations

## Data simulation: Part 2

Contents:
* Start GRASS GIS
* Random Fractal Surfaces
* Random Point Data

Additional Resources:
    
* [
GRASS GIS overview and manual](http://grass.osgeo.org/grass74/manuals/index.html)
    
* [GRASS book](http://www.grassbook.org/)

Tip: While executing this notebook, you can launch the GUI in the command line interface with `g.gui`. As you create rasters here, they will appear in the GUI where you can render them, view other layers, etc.

#### Start GRASS GIS

In [3]:
import os
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass80", "--config", "python_path"], text=True, shell=True).strip()
)

# Import GRASS packages
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("../../../..grassdata", "nc_spm_08_grass7", "HW2_data_simulation")

# Set Region
gs.run_command("g.region", region="rural_1m")

### Random fractal surfaces

First we generate two series of surfaces with dimension 2.9 to 2.01. The series shows creation of a fractal surface.
Register the series in temporal framework, assign the rasters a common color table and create an animation
(feel free to select larger n or a different dimension).
First, we'll create a list of the dimension 2.9 surfaces:

In [43]:
fractal_dem290_series = """fractal_dem_d290.1|1001
fractal_dem_d290.2|1002
fractal_dem_d290.3|1003
fractal_dem_d290.4|1004
fractal_dem_d290.5|1005
fractal_dem_d290.6|1006
fractal_dem_d290.7|1007
fractal_dem_d290.8|1008
fractal_dem_d290.9|1009"""

and a similar one for the dimension 2.01 series:

In [84]:
fractal_dem201_series = """fractal_dem_d201.1|1001
fractal_dem_d201.2|1002
fractal_dem_d201.3|1003
fractal_dem_d201.4|1004
fractal_dem_d201.5|1005
fractal_dem_d201.6|1006
fractal_dem_d201.7|1007
fractal_dem_d201.8|1008
fractal_dem_d201.9|1009"""

Next, we generate the fractal series, register them and visualize, change color table, and run simple analytics.
We'll run watershed analysis on the final surfaces.

#### *Question 1*
Create a fractal surface with `r.surf.fractal` to create a raster named `fractal_dem_d290` with dimension 2.9 and 9 intermediate images. Use `r.info` to find the minimun and maximum values of the surface.

In [76]:
### YOUR CODE HERE

# Answer
gs.run_command("r.surf.fractal", out="fractal_dem_d290",dim="2.9", n="9")
print(gs.read_command("r.info", map="fractal_dem_d290"))

 +----------------------------------------------------------------------------+
 | Map:      fractal_dem_d290               Date: Wed Dec 08 13:54:44 2021    |
 | Mapset:   HW2_data_simulation            Login of Creator: caitl           |
 | Location: nc_spm_08_grass7                                                 |
 | DataBase: C:\Users\caitl\Documents\grassdata                               |
 | Title:                                                                     |
 | Timestamp: none                                                            |
 |----------------------------------------------------------------------------|
 |                                                                            |
 |   Type of Map:  raster               Number of Categories: 0               |
 |   Data Type:    DCELL                Band reference: (none)                |
 |   Rows:         750                                                        |
 |   Columns:      700                  

Now, we'll register each of the intermediate images as a space time raster dataset and compute a few simple statistics.

In [77]:
gs.run_command("t.create", output="fractal_dem_290", type="strds", temporaltype="relative", 
               title="Fractal DEMs with d=2.90", description="Generated data")

# FIND A BETTER SOLUTION HERE: How do we pass a string from python instead of a file to t.register? SO FRUSTRATED THIS DOESN'T WORK?!
#gs.write_command("t.register", input="fractal_dem_290", type="raster", file="-", stdin=fractal_dem290_series, unit="years")

# Tempoary Workaround: create txt file in same directory series names
gs.run_command("t.register", input="fractal_dem_290", type="raster", file="fractal_dem290_series.txt", unit="years")

print(gs.read_command("t.rast.list", input="fractal_dem_290"))

name|mapset|start_time|end_time
fractal_dem_d290.1|HW2_data_simulation|1001|None
fractal_dem_d290.2|HW2_data_simulation|1002|None
fractal_dem_d290.3|HW2_data_simulation|1003|None
fractal_dem_d290.4|HW2_data_simulation|1004|None
fractal_dem_d290.5|HW2_data_simulation|1005|None
fractal_dem_d290.6|HW2_data_simulation|1006|None
fractal_dem_d290.7|HW2_data_simulation|1007|None
fractal_dem_d290.8|HW2_data_simulation|1008|None
fractal_dem_d290.9|HW2_data_simulation|1009|None



In [78]:
rastlist = gs.read_command("t.rast.list", input="fractal_dem_290")
rastlist = str(rastlist)
rastlist = rastlist.split("\r\n")

names=[]

for line in rastlist[1:-1]:
    line = line.split("|")
    names.append(line[0])
print(names)

['fractal_dem_d290.1', 'fractal_dem_d290.2', 'fractal_dem_d290.3', 'fractal_dem_d290.4', 'fractal_dem_d290.5', 'fractal_dem_d290.6', 'fractal_dem_d290.7', 'fractal_dem_d290.8', 'fractal_dem_d290.9']


In [88]:
gs.run_command("t.rast.colors", input="fractal_dem_290", co="elevation")

filenames=[]
for name in names:
    filename="{}.png".format(name)
    filenames.append(filename)
    img = gj.GrassRenderer(filename=filename)
    img.d_rast(map=name)


# https://stackoverflow.com/questions/753190/programmatically-generate-video-or-animated-gif-in-python
from PIL import Image

# filepaths
fp_out = "image.gif"

# https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#gif
img, *imgs = [Image.open(f) for f in sorted(filenames)]
img.save(fp=fp_out, format='GIF', append_images=imgs,
         save_all=True, duration=500, loop=0)
    
from IPython.display import Image
Image(url='image.gif')  
    

In [83]:
# Get some simple univariate statistics
print(gs.read_command("t.rast.univar", input="fractal_dem_290"))

id|start|end|mean|min|max|mean_of_abs|stddev|variance|coeff_var|sum|null_cells|cells|non_null_cells
fractal_dem_d290.1@HW2_data_simulation|1001|None|-28.9745386452122|-1579.77657526412|1400.86701875217|317.51873317571|398.458484333811|158769.163737598|-1375.20217047409|-15211632.7887364|0|525000|525000
fractal_dem_d290.2@HW2_data_simulation|1002|None|-28.8940583232225|-1702.09387319681|1639.41727513369|335.375489674691|420.174954263505|176546.992190339|-1454.19154887566|-15169380.6196918|0|525000|525000
fractal_dem_d290.3@HW2_data_simulation|1003|None|-28.864172598602|-1713.32912037242|1736.11104483667|344.070529251117|430.955455551783|185722.604669845|-1493.04628109331|-15153690.6142661|0|525000|525000
fractal_dem_d290.4@HW2_data_simulation|1004|None|-28.8534848984462|-1836.21551854953|1798.64059644284|349.55115732787|437.703795075601|191584.612223583|-1516.98762425461|-15148079.5716842|0|525000|525000
fractal_dem_d290.5@HW2_data_simulation|1005|None|-28.856021549554|-1907.85538408617

#### *Question 2*
Repeat the process above but for dimension = 2.01. Compute and compare the flow accumulations and drainage basins for both dimensions (2.9 and 2.01). How are they different?

Your Answer Here.

##### Answer:

Same as above but then run and compare:

`r.watershed -a fractal_dem_d290 thresh=15000 accum=f290_accum_15K drain=f290_draindir_15K basin=f290_basin_15K`

`r.watershed -a fractal_dem_d201 thresh=15000 accum=f201_accum_15K drain=f201_draindir_15K basin=f201_basin_15K`

### Surfaces with Fractal Noise

#### *Question 3*
Let's return to the DEMs with stocastic noise that we created in part 1.

Say the lidar error is +/- 30 cm. Rescale the fractal surface to [-0.3,0.3] and add to the `elev_lid792_1m` DEM as a simulated noise. Also compute the flow accumation, drainage direction and basins with a threshold of 15000 (similar to part 1). 

The water flow pattern may represent flow over rough surface, where the roughness is represented in the DEM (e.g. as a grass cover) rather than a constant value (e.g. Mannings coefficient).

Your Answer Here.

#### Answer:
```
r.univar fractal_dem_d290
r.mapcalc "elev_noisefractal = elev_lid792_1m +0.6*(fractal_dem_d290-50.9649)/3754.7"
r.watershed -a elev_noisefractal thresh=15000 accum=_accum_15K_noisefrac drain=draindir_15K_noisefrac basin=basin_15K_noisefrac
```

### Comparing stochastic noise to real LiDAR sensor noise

In addition to the lidar-derived, bare-earth DEM, we have the original lidar point data, `elev_lid792_bepts`. Next, let's compute the elevation surface from point data with different tension parameters and compare the impact on flow modeling.

In [95]:
gs.run_command("g.region", raster="elev_lid792_1m", flags="p")

# First, use the default tension (40)
gs.run_command("v.surf.rst", input="elev_lid792_bepts", elev="elev_lid_default", npmin=120) 

# Higher Tension
gs.run_command("v.surf.rst", input="elev_lid792_bepts", elev="elev_lid_t120", npmin=120, ten=120)

# Compute flow accumulation, drainage direction and basins for comparison to DEMs with stochastic noise
gs.run_command("r.watershed", elevation="elev_lid_default", threshold=5000, accumulation="accum_5K_liddef", drainage="draindir_5K_liddef",
               basin="basin_5K_liddef", flags="a")

gs.run_command("r.watershed", elevation="elev_lid_t120", threshold=5000, accumulation="accum_5K_lidt120", drainage="draindir_5K_lidt120",
               basin="basin_5K_lidt120", flags="a")

#### *Question 4*

Compare the drainage patterns. Which of the surfaces with simulated noise (uniform, gaussian, spatially dependent gaussian and fractal) produce the flow pattern closest to the surface with noise from lidar point cloud (ten=120)? 

###  Random point data 

In this example, we analyze how errors in point position can influence viewshed analysis. First, install r.viewshed.cva addon:

In [ ]:
!g.extension r.viewshed.cva

Generate random points and perturb them. For both points set compute a cumulative viewshed and compare them:

In [99]:
# Generate random points and compute a cumulative viewshed
gs.run_command("v.random", output="random_view", npoints=10, seed=4)
gs.run_command("r.viewshed.cva", input="elev_lid792_1m", vector="random_view", output="cumulative")

#### *Question 5*

Perturb the points with `v.perturb` with parameter = 20 and seed = 1. Computer the cummulative viewshed for the perturbed point and compare the results (in addition to mapping the viewsheds, `r.report` may be useful). What is the impact of error in point position on total viewshed area? 

In [100]:
## ANSWER
# Perturb points and compute cumulative viewshed
gs.run_command("v.perturb", input="random_view", output="random_view_perturb", parameters=20, seed=1)
gs.run_command("r.viewshed.cva", input="elev_lid792_1m", vector="random_view_perturb", output="cumulative_perturb")

Additional commands you can use to simulate simulate point data in raster or vector representation:
r.random, r.random.cells, r.sample.category, v.perturb, v.random, v.kcv 

### Additional Information
* [Website](https://github.com/ncsu-geoforall-lab/geospatial-simulations-course)
    
* [Computing Help](https://help.ncsu.edu/)
    
* [GIST Home](https://geospatial.ncsu.edu/)
    
* [Disclaimer](https://www.ncsu.edu/policies/prr-disclaimer.php)
    
* [Accessibility](https://oit.ncsu.edu/itaccess)

* License: 2018 [CC BY-SA](https://creativecommons.org/licenses/by-sa/4.0/)
  
* [NCSU GeoForAll Lab](https://geospatial.ncsu.edu/geoforall/)